In [4]:
# this script simulates the process of increasing sample size in improving model precision

# although I myself think the idea was terrible...



_EPSILON = 1e-08



from cmath import inf
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import os
import sys
import json
import time as timepackage
import datetime

from sklearn.model_selection import train_test_split

import import_data as impt

from class_DeepLongitudinal import Model_Longitudinal_Attention

from utils_eval             import c_index, brier_score
from utils_log              import save_logging, load_logging
from utils_helper           import f_get_minibatch, f_get_boosted_trainset

sys.argv = ['xx', './params/onlyBL.json']
#time_tag = timepackage.strftime("%d_%b_%Y_%H%M%S%f_GMT", sys_time)
time_tag = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S-%f')

# read in external parameters passed from the console
with open(sys.argv[1]) as p: 
    params = json.load(p)
new_parser = params["new_parser"]
dataset_info = params["dataset_info"]
evaluation_info = params["evaluation_info"]
model_configs = params["model_configs"]
eval_configs = params["eval_configs"]

params["new_parser"]["time_tag"] = time_tag

# In[ ]:


def recursion(dic): 
    for key, value in dic.items(): 
        if isinstance(value, np.floating): 
            dic[key] = float(value)
        elif isinstance(value, dict): 
            recursion(value)


def _f_get_pred(sess, model, data, data_mi, pred_horizon):
    """
        predictions based on the prediction time.
        create new_data and new_mask2 that are available previous or equal to the prediction time (no future measurements are used)
    """
    new_data    = np.zeros(np.shape(data))
    new_data_mi = np.zeros(np.shape(data_mi))

    meas_time = np.concatenate([np.zeros([np.shape(data)[0], 1]), np.cumsum(data[:, :, 0], axis=1)[:, :-1]], axis=1)

    for i in range(np.shape(data)[0]):
        last_meas = np.sum(meas_time[i, :] <= pred_horizon)

        new_data[i, :last_meas, :]    = data[i, :last_meas, :]
        new_data_mi[i, :last_meas, :] = data_mi[i, :last_meas, :]

    return model.predict(new_data, new_data_mi)


def f_get_risk_predictions(sess, model, data_, data_mi_, pred_time, eval_time):
    
    pred = _f_get_pred(sess, model, data_[[0]], data_mi_[[0]], 0)
    _, num_Event, num_Category = np.shape(pred)
       
    risk_all = {}
    for k in range(num_Event):
        risk_all[k] = np.zeros([np.shape(data_)[0], len(pred_time), len(eval_time)])
            
    for p, p_time in enumerate(pred_time):
        ### PREDICTION
        pred_horizon = int(p_time)
        pred = _f_get_pred(sess, model, data_, data_mi_, pred_horizon)


        for t, t_time in enumerate(eval_time):
            eval_horizon = int(t_time) + pred_horizon #if eval_horizon >= num_Category, output the maximum...

            # calculate F(t | x, Y, t >= t_M) = \sum_{t_M <= \tau < t} P(\tau | x, Y, \tau > t_M)
            risk = np.sum(pred[:,:,pred_horizon:(eval_horizon+1)], axis=2) #risk score until eval_time
            risk = risk / (np.sum(np.sum(pred[:,:,pred_horizon:], axis=2), axis=1, keepdims=True) +_EPSILON) #conditioniong on t > t_pred
            
            for k in range(num_Event):
                risk_all[k][:, p, t] = risk[:, k]
                
    return risk_all


# ### 1. Import Dataset
# #####      - Users must prepare dataset in csv format and modify "import_data.py" following our examplar "PBC2"

# In[ ]:


data_mode                   = model_configs["data_mode"]
seed                        = model_configs["seed"]

##### IMPORT DATASET
"""
    num_Category            = max event/censoring time * 1.2
    num_Event               = number of evetns i.e. len(np.unique(label))-1
    max_length              = maximum number of measurements
    x_dim                   = data dimension including delta (1 + num_features)
    x_dim_cont              = dim of continuous features
    x_dim_bin               = dim of binary features
    mask1, mask2, mask3     = used for cause-specific network (FCNet structure)
"""
dirs = dataset_info
test_dir = []
for key in list(dirs.keys()): 
    if key == data_mode: 
        train_dir = dirs[key]
    else: 
        test_dir.append(dirs[key])
    

(tr_x_dim, tr_x_dim_cont, tr_x_dim_bin), (tr_data, tr_time, tr_label), (tr_mask1, tr_mask2, tr_mask3), (tr_data_mi), (tr_id), tr_feat_list = impt.import_dataset(path = train_dir, bin_list_in = model_configs["bin_list"], cont_list_in = model_configs["cont_list"], log_list = model_configs["log_transform"])

(te_x_dim, te_x_dim_cont, te_x_dim_bin), (te_data, te_time, te_label), (te_mask1, te_mask2, te_mask3), (te_data_mi), (te_id), te_feat_list = impt.import_dataset(path = test_dir[0], bin_list_in = model_configs["bin_list"], cont_list_in = model_configs["cont_list"], log_list = model_configs["log_transform"])

(tea_x_dim, tea_x_dim_cont, tea_x_dim_bin), (tea_data, tea_time, tea_label), (tea_mask1, tea_mask2, tea_mask3), (tea_data_mi), (tea_id), tea_feat_list = impt.import_dataset(path = test_dir[1], bin_list_in = model_configs["bin_list"], cont_list_in = model_configs["cont_list"], log_list = model_configs["log_transform"])

pred_time = evaluation_info["pred_time"] # prediction time (in months)
eval_time = evaluation_info["eval_time"] # months evaluation time (for C-index and Brier-Score)


_, num_Event, num_Category  = np.shape(tr_mask1)  # dim of mask3: [subj, Num_Event, Num_Category]

max_length                  = np.shape(tr_data)[1]


mode_path = "{}".format(data_mode)

if not os.path.exists(mode_path):
    os.makedirs(mode_path)

file_path = mode_path + "/" + time_tag + "_" + new_parser["reference"]

if not os.path.exists(file_path):
    os.makedirs(file_path)

# ### 2. Set Hyper-Parameters
# ##### - Play with your own hyper-parameters!

# In[ ]:


burn_in_mode                = model_configs["burnin_mode"]
boost_mode                  = model_configs["boost_mode"]




# INPUT DIMENSIONS
input_dims                  = { "x_dim"         : tr_x_dim,
                                "x_dim_cont"    : tr_x_dim_cont,
                                "x_dim_bin"     : tr_x_dim_bin,
                                "num_Event"     : num_Event,
                                "num_Category"  : num_Category,
                                "max_length"    : max_length }

# NETWORK HYPER-PARMETERS
network_settings            = { "h_dim_RNN"         : new_parser["h_dim_RNN"],
                                "h_dim_FC"          : new_parser["h_dim_FC"],
                                "num_layers_RNN"    : new_parser["num_layers_RNN"],
                                "num_layers_ATT"    : new_parser["num_layers_ATT"],
                                "num_layers_CS"     : new_parser["num_layers_CS"],
                                "RNN_type"          : new_parser["RNN_type"],
                                "FC_active_fn"      : tf.nn.relu,
                                "RNN_active_fn"     : tf.nn.tanh,
                                "initial_W"         : tf.contrib.layers.xavier_initializer(),

                                "reg_W"             : new_parser["reg_W"],
                                "reg_W_out"         : float(new_parser["reg_W_out"])
                                 }


mb_size           = new_parser["mb_size"]
iteration         = new_parser["iteration"]
iteration_burn_in = new_parser["iteration_burn_in"]

keep_prob         = new_parser["keep_prob"]
lr_train          = new_parser["lr_train"]

alpha             = new_parser["alpha"]
beta              = new_parser["beta"]
gamma             = new_parser["gamma"]

# SAVE HYPERPARAMETERS
log_name = file_path + "/" + "_log.json"
log_file = params

recursion(log_file)

with open(log_name, "w") as f:
    json.dump(log_file, f)




In [5]:
tr_data.shape

(2148, 1, 15)

In [ ]:
# ### 3. Split Dataset into Train/Valid/Test Sets

# In[ ]:


(tr_data,va_data, tr_data_mi, va_data_mi, tr_time,va_time, tr_label,va_label, 
 tr_mask1,va_mask1, tr_mask2,va_mask2, tr_mask3,va_mask3) = train_test_split(tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3, test_size=model_configs["val_ratio"], random_state = seed) 

if boost_mode == "ON":
    tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3 = f_get_boosted_trainset(tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3)
    
    